In [1]:
import psycopg2

# Function to read the db connection info
def read_db_connection_info(filename="../../db_connection_info.txt"):
    connection_info = {}
    with open(filename, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            connection_info[key] = value
    return connection_info

# Load the connection info
connection_info = read_db_connection_info()

try:
    # Connect to the database
    conn = psycopg2.connect(
        host=connection_info["host"],
        user=connection_info["user"],
        password=connection_info["password"],
        dbname=connection_info["database"],
        port=connection_info["port"]
    )
    cursor = conn.cursor()

    
    # Query to find out how many ships each type of thing was used on
    query_ships_per_property = """
    SELECT thing, property, COUNT(DISTINCT ships_idx) AS ships_count
    FROM data_mapping
    WHERE ships_idx BETWEEN 1000 AND 1900
    AND thing <> 'ISS.AMS.DataGroup01.T' 
    GROUP BY thing, property
    HAVING COUNT(DISTINCT ships_idx) >= 5
    ORDER BY ships_count DESC, thing, property;
    """
    cursor.execute(query_ships_per_property)  # Assuming query_ships_per_thing contains the updated SQL query

    # Fetch and print the result
    ships_per_property = cursor.fetchall()
    # Print the total number of properties
    print("Total number of properties:", len(ships_per_property))
    
    # Print details for each property
    print("Details of each property used on 3 or more ships:")
    for thing, property, ships_count in ships_per_property:
        print(f"Thing: {thing}, Property: {property}, Number of Ships: {ships_count}")

    

    # Close the cursor and connection
    cursor.close()
    conn.close()
    
except Exception as e:
    print(f"An error occurred: {e}")


Total number of properties: 325
Details of each property used on 3 or more ships:
Thing: DE2Thing, Property: FOInletTemp, Number of Ships: 50
Thing: DE1Thing, Property: FOInletTemp, Number of Ships: 49
Thing: DE2Thing, Property: Cy1ExhGasOutletTemp, Number of Ships: 49
Thing: DE2Thing, Property: Cy2ExhGasOutletTemp, Number of Ships: 49
Thing: DE2Thing, Property: Cy3ExhGasOutletTemp, Number of Ships: 49
Thing: DE2Thing, Property: Cy4ExhGasOutletTemp, Number of Ships: 49
Thing: DE2Thing, Property: Cy5ExhGasOutletTemp, Number of Ships: 49
Thing: DE2Thing, Property: RPM, Number of Ships: 49
Thing: DE3Thing, Property: FOInletTemp, Number of Ships: 49
Thing: DG1Thing, Property: WindingTempR, Number of Ships: 49
Thing: DG1Thing, Property: WindingTempS, Number of Ships: 49
Thing: DG1Thing, Property: WindingTempT, Number of Ships: 49
Thing: DG2Thing, Property: WindingTempR, Number of Ships: 49
Thing: DG2Thing, Property: WindingTempS, Number of Ships: 49
Thing: DG2Thing, Property: WindingTempT, 

In [2]:
import re

# Assuming ships_per_property is your list of tuples [(thing, property, ships_count), ...]

# Initialize an empty dictionary to hold the aggregated counts
aggregated_data = {}

for thing, property, ships_count in ships_per_property:
    # Replace all digits in the thing and property names with '#'
    modified_thing = re.sub(r'\d+', '#', thing)
    modified_property = re.sub(r'\d+', '#', property)

    # Initialize nested dictionary if the modified thing does not exist
    if modified_thing not in aggregated_data:
        aggregated_data[modified_thing] = {}

    # Aggregate ship counts by modified property under each modified thing
    if modified_property in aggregated_data[modified_thing]:
        aggregated_data[modified_thing][modified_property] += ships_count
    else:
        aggregated_data[modified_thing][modified_property] = ships_count

# Now, print the aggregated results for properties with ship_count > 5
for modified_thing in aggregated_data:    
    for modified_property, ships_count in aggregated_data[modified_thing].items():
        if ships_count >= 10:  # Check if ships_count is more than 5
            print(f"{modified_thing}, {modified_property}, {ships_count}")


DE#Thing, FOInletTemp, 154
DE#Thing, Cy#ExhGasOutletTemp, 1047
DE#Thing, RPM, 150
DE#Thing, CFWOutletTemp, 150
DE#Thing, FOInletPress, 150
DE#Thing, ScavTemp, 150
DE#Thing, ScavPress, 146
DE#Thing, CFWInletPress, 141
DE#Thing, CFWInletTemp, 138
DE#Thing, LOInletTemp, 133
DE#Thing, LOInletPress, 129
DE#Thing, LTInletPress, 99
DE#Thing, DoUse, 55
DE#Thing, LTInletTemp, 90
DE#Thing, FOFilterInletPress, 81
DE#Thing, LOFilterInletPress, 83
DE#Thing, LTOutletTemp, 72
DE#Thing, SAPress, 66
DE#Thing, LFOUse, 24
DE#Thing, FGMassFlow, 22
DE#Thing, GasMode, 17
DE#Thing, FGPressure, 16
DE#Thing, FGTemperature, 16
DE#Thing, Load, 10
DG#Thing, WindingTempR, 152
DG#Thing, WindingTempS, 152
DG#Thing, WindingTempT, 152
DG#Thing, Voltage, 137
DG#Thing, GenPower, 135
DG#Thing, Frequency, 133
DG#Thing, Current, 130
DG#Thing, RunningState, 130
DG#Thing, busBarVoltage, 39
DG#Thing, CBClosed, 38
DG#Thing, Load, 37
DG#Thing, busBarFrequency, 31
DG#Thing, CurrentB, 10
DG#Thing, FrequencyA, 10
DG#Thing, Frequen

In [3]:
import csv

# Write the filtered aggregated results to a CSV file
with open('master_model.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Write the header
    csvwriter.writerow(['thing', 'property', 'ships_count'])

    # Write the data
    for modified_thing in aggregated_data:    
        for modified_property, ships_count in aggregated_data[modified_thing].items():
            csvwriter.writerow([modified_thing, modified_property, ships_count])
